In [ ]:
---
toc: true
layout: post
description: RQ-Decomposition-In-Practice
categories: [Computer Vision,Linear Algebra]
---

Let's keep things short and sweet.

Given a camera projection matrix, $P$, we can decompose it into a $K$ (Camera Matrix), $R$ (Rotation Matrix) and $C$ (Camera Centroid) matrix.

IE, given we have  $P = K[R|-RC]$, We want to find $K$, $R$ and $C$.

The method is simply described in *Multiple View Geometry in Computer Vision (Second Edition)*, on page 163, however let's turn it into a practical *Python* implementation.

Let's follow along with an example from the book.

In [1]:
import numpy as np
from scipy import linalg
np.set_printoptions(precision=2)

In [2]:
P = np.array([[3.53553e+2,  3.39645e+2, 2.77744e+2, -1.44946e+6],
              [-1.03528e+2, 2.33212e+1, 4.59607e+2, -6.3252e+5],
              [7.07107e-1, -3.53553e-1, 6.12372e-1, -9.18559e+2]])

So, we have: $P = [M | −MC]$

M can be decomposed as $M=KR$ using the RQ decomposition.

```python
M = P[0:3,0:3]
K, R = linalg.rq(M)
```

So far, so good.

Now things get a little more complex. 

We want to find a Camera matrix with a positive diagonal, giving positive focal lengths.

However, in case this doesn't happen, we can adjust the sign of the column of each column of both the $K$ and $R$ matrix, to "Make it so".



```python
T = np.diag(np.sign(np.diag(K)))
if linalg.det(T) < 0:
    T[1,1] *= -1

K = np.dot(K,T)
R = np.dot(T,R)
```

    
Finally, we can find the Camera Center ($C$).

We have $P_4$, the 4th column of $P$.


$P_4 = −MC$


From this, we can find $C = {-M}^{-1} P_4$

def factorize(P):
    M = P[0:3,0:3]
    
    K,R = linalg.rq(M)
    
    T = np.diag(np.sign(np.diag(K)))
    
    if linalg.det(T) < 0:
        T[1,1] *= -1
    
    K = np.dot(K,T)
    R = np.dot(T,R)
    
    C = np.dot(linalg.inv(-M),P[:,3])
    return(K,R,C)


    
K,R,C = factorize(P)

print('K')
print(K)

print('R')
print(R)

print('C')
print(C)


Voila!

[This](https://www.uio.no/studier/emner/matnat/its/nedlagte-emner/UNIK4690/v17/forelesninger/lecture_5_2_pose_from_known_3d_points.pdf) presentation is a great read, and provides a good overview of the RQ and QR decompositions.